# COGS 108 - Data Checkpoint

# Names

- Jackson Conte
- Andrew Nguyen
- Anish Rajeshkumar
- Baraa Zekeria

<a id='research_question'></a>
# Research Question

<!--**Fill in your research question here**-->

Is there a correlation between the proportion of the type of shot (e.g. fieldgoal, 3-point, free throw) and playoff success (i.e. advancement into the playoffs) in the past 20 years for the National Basketball Association (NBA)?

# Dataset(s)

<!--*Fill in your dataset information here*-->

<!--*(Copy this information for each dataset)*-->
- Dataset Name: National Basketball Association (NBA) Statistics
- Link to the dataset:
    - [nba_api](https://github.com/swar/nba_api): ```General website (API)```
        - [teamyearbyyearstats](https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/teamyearbyyearstats.md): ```Dataset (Endpoint)```
        - [teams.py](https://github.com/swar/nba_api/blob/master/src/nba_api/stats/static/players.py): ```Helper script```
- Number of observations: ```1,597```

<!--*1-2 sentences describing each dataset.*-->

The ```teamyearbyyearstats``` endpoint has records by season for each NBA team from the 1949 season. To extract each team's data from the endpoint, their respective ID is called from ```teams.py``` and the endpoint outputs the respective ```DataFrame()```.

<!--*If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.*-->

# Setup

In [1]:
## YOUR CODE HERE

# upgrade pip
!{sys.executable} -m pip install --upgrade pip

#install external libraries
import sys
!{sys.executable} -m pip install nba_api # nba stats
!{sys.executable} -m pip install pandera # data validation 

# libraries
import pandas as pd
import numpy as np
from pathlib import Path
import pandera as pa
from pandera import Check, Column, DataFrameSchema, DateTime, check
import datetime as dt

# notebook configurations
pd.options.display.max_colwidth = 100

# NBA API libraries
## team info
from nba_api.stats.static import teams
## team stats
from nba_api.stats.endpoints import teamyearbyyearstats as teamyears

print("\n**ALL LIBRARIES IMPORTED SUCCESSFULLY**")

/bin/bash: {sys.executable}: command not found
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable

**ALL LIBRARIES IMPORTED SUCCESSFULLY**


/opt/conda/lib/python3.9/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/opt/conda/lib/python3.9/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/opt/conda/lib/python3.9/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


# Data Cleaning

<!--*Describe your data cleaning steps here.*-->

1. **API**
    - Call the ```teamyearbyyearstats``` endpoint from the ```nba_api``` by each team's ID to output the dataframe (DF)
        - Combine all of the DFs into one DF
            - Save DF as ```nba_stats_raw.csv```
1. **Validation**
    - Subset the DF by columns of interest
    - Create test cases (e.g. data type constraints, range constraints, cross-field validation) for each column via the ```pandera``` library
        - Null test cases is accounted for automatically 
    - Validate the tests
        - Output the schema errors and DF errors into a seperate DF (e.g. ```schema_errors```, ```df_errors```)
1. **Cleaning**
    - Fix the respective test cases that failed
1. **Verification**
    - Validate the tests
        - Output the schema errors and DF errors into a seperate DF (e.g. ```schema_errors```, ```df_errors```)
1. **Export Data**
    - Save DF as ```nba_stats_cleaned.csv```
1. **Data Dictionary**
    - Create a data dictionary for the ```nba_stats_cleaned.csv``` dataset

In [2]:
import time
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
team_ids = [t['id'] for t in teams.get_teams()]
df = pd.DataFrame()

# slow but effective
for i in team_ids:
    df = pd.concat([df, teamyears.TeamYearByYearStats(i).get_data_frames()[0]])
    time.sleep(0.4) # any lower and it'll time out
print(df)

       TEAM_ID   TEAM_CITY   TEAM_NAME     YEAR  GP  WINS  LOSSES  WIN_PCT  \
0   1610612737  Tri-Cities  Blackhawks  1949-50  64    29      35    0.453   
1   1610612737  Tri-Cities  Blackhawks  1950-51  68    25      43    0.368   
2   1610612737   Milwaukee       Hawks  1951-52  66    17      49    0.258   
3   1610612737   Milwaukee       Hawks  1952-53  71    27      44    0.380   
4   1610612737   Milwaukee       Hawks  1953-54  72    21      51    0.292   
..         ...         ...         ...      ...  ..   ...     ...      ...   
28  1610612766   Charlotte     Hornets  2018-19  82    39      43    0.476   
29  1610612766   Charlotte     Hornets  2019-20  65    23      42    0.354   
30  1610612766   Charlotte     Hornets  2020-21  72    33      39    0.458   
31  1610612766   Charlotte     Hornets  2021-22  82    43      39    0.524   
32  1610612766   Charlotte     Hornets  2022-23   9     3       6    0.333   

    CONF_RANK  DIV_RANK  ...  OREB  DREB   REB   AST    PF  STL

In [3]:
df.sample(5)

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
49,1610612745,Houston,Rockets,2016-17,82,55,27,0.671,3,2,...,892,2746,3638,2070,1634,670,1240,352,9458,2
28,1610612753,Orlando,Magic,2017-18,82,25,57,0.305,14,4,...,722,2692,3414,1921,1579,622,1192,400,8479,24
6,1610612739,Cleveland,Cavaliers,1976-77,82,43,39,0.524,6,4,...,1312,2563,3875,1845,1951,579,1356,472,8370,20
46,1610612741,Chicago,Bulls,2012-13,82,45,37,0.549,5,2,...,1026,2514,3540,1886,1617,588,1171,417,7641,29
42,1610612764,Washington,Wizards,2003-04,82,25,57,0.305,13,6,...,1115,2392,3507,1537,1712,733,1439,406,7528,18


In [4]:
df.to_csv("data/nba_stats_raw.csv", index = False)

## Validation

In [5]:
df.columns

Index(['TEAM_ID', 'TEAM_CITY', 'TEAM_NAME', 'YEAR', 'GP', 'WINS', 'LOSSES',
       'WIN_PCT', 'CONF_RANK', 'DIV_RANK', 'PO_WINS', 'PO_LOSSES',
       'CONF_COUNT', 'DIV_COUNT', 'NBA_FINALS_APPEARANCE', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'PF', 'STL', 'TOV', 'BLK', 'PTS', 'PTS_RANK'],
      dtype='object')

In [6]:
df = df[["TEAM_ID", "TEAM_CITY", "TEAM_NAME", "YEAR", "GP", "WINS", "LOSSES", "WIN_PCT", 
         "CONF_RANK", "DIV_RANK", "PO_WINS", "PO_LOSSES", "NBA_FINALS_APPEARANCE", 
         "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "PTS", "PTS_RANK"]]
df.columns

Index(['TEAM_ID', 'TEAM_CITY', 'TEAM_NAME', 'YEAR', 'GP', 'WINS', 'LOSSES',
       'WIN_PCT', 'CONF_RANK', 'DIV_RANK', 'PO_WINS', 'PO_LOSSES',
       'NBA_FINALS_APPEARANCE', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'PTS', 'PTS_RANK'],
      dtype='object')

In [7]:
team_nickname = [t["nickname"] for t in teams.get_teams()]

In [8]:
schema = pa.DataFrameSchema(
    {
        "TEAM_ID": Column(int, Check.isin(team_ids)),
        "TEAM_CITY": Column(str),
        "TEAM_NAME": Column(str, Check.isin(team_nickname)),
        "YEAR": Column(int, Check.greater_than(1999), Check.less_than(2022)),
        "GP": Column(int, Check.greater_than(0), Check.less_than(83)),
        "WINS": Column(int, Check.greater_than(-1), Check.less_than(83)),
        "LOSSES": Column(int, Check.greater_than(-1), Check.less_than(83)),
        "WIN_PCT": Column(float, Check.greater_than(0), Check.less_than(1.0)),
        "CONF_RANK": Column(int),
        "DIV_RANK": Column(int),
        "PO_WINS": Column(int, Check.greater_than(-1), Check.less_than(29)),
        "PO_LOSSES": Column(int, Check.greater_than(-1), Check.less_than(29)),
        "NBA_FINALS_APPEARANCE": Column(str),
        "FGM": Column(int, Check.greater_than(-1)),
        "FGA": Column(int, Check.greater_than(-1)),
        "FG_PCT": Column(float, Check.greater_than(0), Check.less_than(1.0), lambda x: x == (x["FGM"] / x["FGA"])),
        "FG3M": Column(int, Check.greater_than(-1)),
        "FG3A": Column(int, Check.greater_than(-1)),
        "FG3_PCT": Column(float, Check.greater_than(0), Check.less_than(1.0), lambda x: x == (x["FG3M"] / x["FG3A"])),
        "FTM": Column(int, Check.greater_than(-1)),
        "FTA": Column(int, Check.greater_than(-1)),
        "FT_PCT": Column(float, Check.greater_than(0), Check.less_than(1.0), lambda x: x == (x["FTM"] / x["FTA"])),
        "PTS": Column(int),
        "PTS_RANK": Column(int, Check.greater_than(0), Check.less_than(31)),   
    }
)

In [9]:
schema_errors = pd.DataFrame()
df_errors = pd.DataFrame()

try:
    schema.validate(df, lazy=True)
    print("All Tests Passed")
except pa.errors.SchemaErrors as err:
    # schema errors and failure cases
    schema_errors = err.failure_cases
    # DataFrame object that failed validation
    df_errors = err.data
    print("--- " + str(len(schema_errors["check"])), "Test(s) Failed ---")
    if "nullable" not in list(schema_errors["check"].unique()):
        print("--- Hidden Test Passed: No Null Values ---")
    else:
        print("--- Hidden Test Failed: Null Values ---")

--- 3704 Test(s) Failed ---
--- Hidden Test Passed: No Null Values ---


In [10]:
schema_errors

,schema_context,column,check,check_number,failure_case,index
0,Column,TEAM_NAME,"isin({'Bucks', 'Clippers', 'Lakers', 'Knicks', 'Rockets', 'Jazz', 'Cavaliers', 'Nets', 'Maverick...",0,Blackhawks,0
3792,Column,FG3_PCT,greater_than(0),0,0.0,14
3812,Column,FG3_PCT,greater_than(0),0,0.0,10
3811,Column,FG3_PCT,greater_than(0),0,0.0,9
3810,Column,FG3_PCT,greater_than(0),0,0.0,8
...,...,...,...,...,...,...
1543,Column,FG_PCT,field_uniqueness,None,0.461,26
1542,Column,FG_PCT,field_uniqueness,None,0.467,25
1541,Column,FG_PCT,field_uniqueness,None,0.468,24
1783,Column,FG_PCT,greater_than(0),0,0.0,35


In [11]:
df_errors

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,PTS,PTS_RANK
0,1610612737,Tri-Cities,Blackhawks,1949-50,64,29,35,0.453,0,3,...,0,0.000,0,0,0.000,0,2308,0.727,5313,10
1,1610612737,Tri-Cities,Blackhawks,1950-51,68,25,43,0.368,0,5,...,0,0.000,0,0,0.000,0,2425,0.723,5730,3
2,1610612737,Milwaukee,Hawks,1951-52,66,17,49,0.258,0,5,...,0,0.000,0,0,0.000,0,2177,0.682,4833,10
3,1610612737,Milwaukee,Hawks,1952-53,71,27,44,0.380,0,5,...,0,0.000,0,0,0.000,0,2400,0.685,5389,9
4,1610612737,Milwaukee,Hawks,1953-54,72,21,51,0.292,0,4,...,0,0.000,0,0,0.000,0,2205,0.691,5038,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,1610612766,Charlotte,Hornets,2018-19,82,39,43,0.476,9,2,...,7362,0.448,977,2783,0.351,1510,1895,0.797,9081,19
29,1610612766,Charlotte,Hornets,2019-20,65,23,42,0.354,10,4,...,5586,0.434,785,2231,0.352,1052,1406,0.748,6687,30
30,1610612766,Charlotte,Hornets,2020-21,72,33,39,0.458,10,4,...,6324,0.455,985,2666,0.369,1146,1505,0.761,7881,23
31,1610612766,Charlotte,Hornets,2021-22,82,43,39,0.524,10,3,...,7497,0.468,1143,3130,0.365,1298,1753,0.740,9457,4


In [12]:
print("-------Schema Errors Summary--------\n")

print("- Failure Cases:\n\n")
for count, check in enumerate(schema_errors["check"].unique()):
    print(str(count + 1) + ".", check, "\n\n")
    
print("- Total Errors:", df_errors.shape[0])

-------Schema Errors Summary--------

- Failure Cases:


1. isin({'Bucks', 'Clippers', 'Lakers', 'Knicks', 'Rockets', 'Jazz', 'Cavaliers', 'Nets', 'Mavericks', 'Trail Blazers', 'Spurs', 'Nuggets', 'Heat', 'Hornets', 'Warriors', 'Hawks', 'Timberwolves', 'Raptors', 'Celtics', 'Pistons', 'Bulls', '76ers', 'Thunder', 'Grizzlies', 'Magic', 'Pacers', 'Suns', 'Pelicans', 'Kings', 'Wizards'}) 


2. greater_than(0) 


3. field_uniqueness 


4. greater_than(1999) 


5. dtype('int64') 


- Total Errors: 1597


## Cleaning

### Schema Test Failures

In [13]:
schema_errors.groupby(["column", "check"]).count()[["schema_context"]]

schema_context
column    check                                                                                                               
FG3_PCT   field_uniqueness                                                                                                1038
          greater_than(0)                                                                                                   36
FG_PCT    field_uniqueness                                                                                                1054
          greater_than(0)                                                                                                   36
FT_PCT    field_uniqueness                                                                                                1403
TEAM_NAME isin({'Bucks', 'Clippers', 'Lakers', 'Knicks', 'Rockets', 'Jazz', 'Cavaliers', 'Nets', 'Mavericks...             135
YEAR      dtype('int64')                                                                                                     1
          greater_than(1999)                                                                                                 1

#### ```dtype('int64')```

In [14]:
# extract first year, convert year to int
df['YEAR'] = df['YEAR'].str[0:4].astype(int)
df

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,PTS,PTS_RANK
0,1610612737,Tri-Cities,Blackhawks,1949,64,29,35,0.453,0,3,...,0,0.000,0,0,0.000,0,2308,0.727,5313,10
1,1610612737,Tri-Cities,Blackhawks,1950,68,25,43,0.368,0,5,...,0,0.000,0,0,0.000,0,2425,0.723,5730,3
2,1610612737,Milwaukee,Hawks,1951,66,17,49,0.258,0,5,...,0,0.000,0,0,0.000,0,2177,0.682,4833,10
3,1610612737,Milwaukee,Hawks,1952,71,27,44,0.380,0,5,...,0,0.000,0,0,0.000,0,2400,0.685,5389,9
4,1610612737,Milwaukee,Hawks,1953,72,21,51,0.292,0,4,...,0,0.000,0,0,0.000,0,2205,0.691,5038,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,1610612766,Charlotte,Hornets,2018,82,39,43,0.476,9,2,...,7362,0.448,977,2783,0.351,1510,1895,0.797,9081,19
29,1610612766,Charlotte,Hornets,2019,65,23,42,0.354,10,4,...,5586,0.434,785,2231,0.352,1052,1406,0.748,6687,30
30,1610612766,Charlotte,Hornets,2020,72,33,39,0.458,10,4,...,6324,0.455,985,2666,0.369,1146,1505,0.761,7881,23
31,1610612766,Charlotte,Hornets,2021,82,43,39,0.524,10,3,...,7497,0.468,1143,3130,0.365,1298,1753,0.740,9457,4


#### ```greater_than(1999)```

In [15]:
# filter for seasons 2000-2021
df = df[(df['YEAR'] >= 2000) & (df['YEAR'] < 2022)]
df

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,PTS,PTS_RANK
51,1610612737,Atlanta,Hawks,2000,82,25,57,0.305,13,7,...,6668,0.431,333,933,0.357,1374,1811,0.759,7459,26
52,1610612737,Atlanta,Hawks,2001,82,33,49,0.402,12,6,...,6610,0.439,423,1194,0.354,1486,1942,0.765,7711,19
53,1610612737,Atlanta,Hawks,2002,82,35,47,0.427,11,5,...,6434,0.444,402,1141,0.352,1594,2011,0.793,7714,18
54,1610612737,Atlanta,Hawks,2003,82,28,54,0.341,12,7,...,6529,0.433,419,1249,0.335,1534,1976,0.776,7611,15
55,1610612737,Atlanta,Hawks,2004,82,13,69,0.159,15,5,...,6672,0.441,304,973,0.312,1417,1994,0.711,7605,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,1610612766,Charlotte,Hornets,2017,82,36,46,0.439,10,3,...,7106,0.450,824,2233,0.369,1656,2216,0.747,8874,10
28,1610612766,Charlotte,Hornets,2018,82,39,43,0.476,9,2,...,7362,0.448,977,2783,0.351,1510,1895,0.797,9081,19
29,1610612766,Charlotte,Hornets,2019,65,23,42,0.354,10,4,...,5586,0.434,785,2231,0.352,1052,1406,0.748,6687,30
30,1610612766,Charlotte,Hornets,2020,72,33,39,0.458,10,4,...,6324,0.455,985,2666,0.369,1146,1505,0.761,7881,23


#### ```isin(team_names)```

In [16]:
df['TEAM_NAME'].unique()

array(['Hawks', 'Celtics', 'Cavaliers', 'Hornets', 'Pelicans', 'Bulls',
       'Mavericks', 'Nuggets', 'Warriors', 'Rockets', 'Clippers',
       'Lakers', 'Heat', 'Bucks', 'Timberwolves', 'Nets', 'Knicks',
       'Magic', 'Pacers', '76ers', 'Suns', 'Trail Blazers', 'Kings',
       'Spurs', 'SuperSonics', 'Thunder', 'Raptors', 'Jazz', 'Grizzlies',
       'Wizards', 'Pistons', 'Bobcats'], dtype=object)

In [17]:
df.loc[df['TEAM_NAME'] == 'Bobcats']

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,PTS,PTS_RANK
14,1610612766,Charlotte,Bobcats,2004,82,18,64,0.220,14,4,...,6859,0.432,320,881,0.363,1487,2096,0.709,7729,22
15,1610612766,Charlotte,Bobcats,2005,82,26,56,0.317,13,4,...,6843,0.433,428,1261,0.339,1593,2186,0.729,7943,18
16,1610612766,Charlotte,Bobcats,2006,82,33,49,0.402,11,4,...,6643,0.446,457,1280,0.357,1568,2136,0.734,7945,18
17,1610612766,Charlotte,Bobcats,2007,82,32,50,0.390,12,4,...,6554,0.452,529,1443,0.367,1510,2115,0.714,7959,19
18,1610612766,Charlotte,Bobcats,2008,82,35,47,0.427,10,4,...,6299,0.455,490,1339,0.366,1455,1965,0.740,7677,30
19,1610612766,Charlotte,Bobcats,2009,82,44,38,0.537,7,4,...,6306,0.453,460,1331,0.346,1637,2180,0.751,7813,28
20,1610612766,Charlotte,Bobcats,2010,82,34,48,0.415,10,4,...,6365,0.451,393,1203,0.327,1511,1999,0.756,7650,29
21,1610612766,Charlotte,Bobcats,2011,66,7,59,0.106,15,5,...,5293,0.414,263,892,0.295,1090,1462,0.746,5739,30
22,1610612766,Charlotte,Bobcats,2012,82,21,61,0.256,14,4,...,6649,0.425,469,1399,0.335,1546,2060,0.750,7661,26
23,1610612766,Charlotte,Bobcats,2013,82,43,39,0.524,7,3,...,6730,0.442,516,1471,0.351,1474,2000,0.737,7942,23


In [18]:
df = df.replace(to_replace="Bobcats",
           value="Hornets")
df = df.replace(to_replace="SuperSonics",
           value="Thunder")
df = df.replace(to_replace="Seattle",
           value="Oklahoma City")
df.loc[df['TEAM_NAME'] == 'Thunder']

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,PTS,PTS_RANK
33,1610612760,Oklahoma City,Thunder,2000,82,44,38,0.537,10,5,...,6649,0.456,466,1169,0.399,1454,1986,0.732,7978,8
34,1610612760,Oklahoma City,Thunder,2001,82,45,37,0.549,7,4,...,6681,0.469,489,1292,0.378,1263,1672,0.755,8014,6
35,1610612760,Oklahoma City,Thunder,2002,82,40,42,0.488,10,5,...,6599,0.437,456,1291,0.353,1325,1780,0.744,7555,23
36,1610612760,Oklahoma City,Thunder,2003,82,37,45,0.451,12,5,...,6590,0.446,723,1936,0.373,1363,1782,0.765,7964,6
37,1610612760,Oklahoma City,Thunder,2004,82,52,30,0.634,4,1,...,6498,0.444,666,1824,0.365,1683,2131,0.790,8113,11
38,1610612760,Oklahoma City,Thunder,2005,82,35,47,0.427,11,3,...,6711,0.459,605,1631,0.371,1652,2104,0.785,8411,2
39,1610612760,Oklahoma City,Thunder,2006,82,31,51,0.378,14,5,...,6628,0.460,525,1451,0.362,1511,1911,0.791,8130,12
40,1610612760,Oklahoma City,Thunder,2007,82,20,62,0.244,15,5,...,7032,0.444,313,939,0.333,1436,1866,0.770,7999,16
41,1610612760,Oklahoma City,Thunder,2008,82,23,59,0.280,13,5,...,6716,0.447,328,949,0.346,1626,2069,0.786,7952,24
42,1610612760,Oklahoma City,Thunder,2009,82,50,32,0.610,8,4,...,6629,0.462,418,1229,0.340,1778,2210,0.805,8322,14


### Categorical Columns

In [19]:
df.select_dtypes(include = "object")

,TEAM_CITY,TEAM_NAME,NBA_FINALS_APPEARANCE
51,Atlanta,Hawks,N/A
52,Atlanta,Hawks,N/A
53,Atlanta,Hawks,N/A
54,Atlanta,Hawks,N/A
55,Atlanta,Hawks,N/A
...,...,...,...
27,Charlotte,Hornets,N/A
28,Charlotte,Hornets,N/A
29,Charlotte,Hornets,N/A
30,Charlotte,Hornets,N/A


```NBA_FINALS_APPEARANCE```

In [20]:
df["NBA_FINALS_APPEARANCE"].unique()

array(['N/A', 'LEAGUE CHAMPION', 'FINALS APPEARANCE'], dtype=object)

No further cleaning needs to be done as ```N/A``` is a correct indicator for ```null``` values (expected for this column).

## Verification

In [21]:
schema_errors = pd.DataFrame()
df_errors = pd.DataFrame()

try:
    schema.validate(df, lazy=True)
    print("All Tests Passed")
except pa.errors.SchemaErrors as err:
    # schema errors and failure cases
    schema_errors = err.failure_cases
    # DataFrame object that failed validation
    df_errors = err.data
    print("--- " + str(len(schema_errors["check"].unique())), "Test(s) Failed ---")
    if "nullable" not in list(schema_errors["check"].unique()):
        print("--- Hidden Test Passed: No Null Values ---")
    else:
        print("--- Hidden Test Failed: Null Values ---")

--- 1 Test(s) Failed ---
--- Hidden Test Passed: No Null Values ---


In [22]:
schema_errors.groupby(["column", "check"]).count()[["schema_context"]]

,,schema_context
column,check,
FG3_PCT,field_uniqueness,591
FG_PCT,field_uniqueness,594
FT_PCT,field_uniqueness,600


In [23]:
schema_errors["check"].unique()[0]

'field_uniqueness'

The failed test above has no importance to our ideal dataframe.

## Export Data

In [24]:
df = df.reset_index(drop = True)

In [25]:
df.to_csv("data/nba_stats_cleaned.csv", index = False)
df

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,PTS,PTS_RANK
0,1610612737,Atlanta,Hawks,2000,82,25,57,0.305,13,7,...,6668,0.431,333,933,0.357,1374,1811,0.759,7459,26
1,1610612737,Atlanta,Hawks,2001,82,33,49,0.402,12,6,...,6610,0.439,423,1194,0.354,1486,1942,0.765,7711,19
2,1610612737,Atlanta,Hawks,2002,82,35,47,0.427,11,5,...,6434,0.444,402,1141,0.352,1594,2011,0.793,7714,18
3,1610612737,Atlanta,Hawks,2003,82,28,54,0.341,12,7,...,6529,0.433,419,1249,0.335,1534,1976,0.776,7611,15
4,1610612737,Atlanta,Hawks,2004,82,13,69,0.159,15,5,...,6672,0.441,304,973,0.312,1417,1994,0.711,7605,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,1610612766,Charlotte,Hornets,2017,82,36,46,0.439,10,3,...,7106,0.450,824,2233,0.369,1656,2216,0.747,8874,10
652,1610612766,Charlotte,Hornets,2018,82,39,43,0.476,9,2,...,7362,0.448,977,2783,0.351,1510,1895,0.797,9081,19
653,1610612766,Charlotte,Hornets,2019,65,23,42,0.354,10,4,...,5586,0.434,785,2231,0.352,1052,1406,0.748,6687,30
654,1610612766,Charlotte,Hornets,2020,72,33,39,0.458,10,4,...,6324,0.455,985,2666,0.369,1146,1505,0.761,7881,23


## Data Dictionary

In [26]:
data_dictionary = pd.read_csv("data/nba_stats_data_dictionary.csv")
data_dictionary

,column_name,column_label,description,data_type,constraint,unique_values,formula
0,TEAM_ID,Team ID,Team ID,int,"PRIMARY KEY, NOT NULL",Yes,NaN
1,TEAM_CITY,Team City,Team city,str,NOT NULL,Yes,NaN
2,TEAM_NAME,Team Name,Team name,str,NOT NULL,Yes,NaN
3,YEAR,Year,NBA season,int,NOT NULL,Yes,NaN
4,GP,Games Played,The number of games played,int,NOT NULL,Yes,NaN
5,WINS,Wins,The number of games won by a player or team,int,NOT NULL,Yes,NaN
6,LOSSES,Losses,The number of games lost by a player or team,int,NOT NULL,Yes,NaN
7,WIN_PCT,Win Percentage,The percentage of games played that a player or team has won,float,NOT NULL,Yes,(WINS) / (GP)
8,CONF_RANK,Conference Rank,Conference rank,int,NOT NULL,Yes,NaN
9,DIV_RANK,Division Rank,Division rank,int,NOT NULL,Yes,NaN
